In [1]:
from utils import *
from tools import *
import pandas as pd

# requirements

In [3]:
chr1 = "../../data/grch38/chromosomes/chr1.fa"
sequence = import_fasta(chr1)


In [4]:
# CLASH

clash_df = preprocess_clash_data()
clash_df = get_ensembl60_data(clash_df)

targetscan_df = pd.read_csv(
    "../data/supplementary_files/targetscan.tsv", sep="\t")

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.pep.all.fa.gz.pickle


In [4]:
def modded_run_script(sequence):
    """runs the script on input sequence string

    Args:
        sequence (str): sequence of interest
    """
    
    # loading data dfs
    targetscan_df = pd.read_csv("../data/supplementary_files/targetscan.tsv", sep="\t")
    ta_sps_df = pd.read_csv("../data/supplementary_files/ta_sps.tsv", sep="\t")
    
    
    results_df = find_matches(sequence, targetscan_df)
    generate_ta_column(results_df, ta_sps_df)
    # generate_sps_column(results_df, ta_sps_df)
    generate_3utr_length_column(sequence, results_df)
    generate_3_supplementary_pairing_column(sequence, results_df)
    # generate_3_supplementary_pairing_column(sequence, results_df, extended=True)
    generate_avg_position_column(results_df)
    generate_close_proximity_column(results_df)
    generate_total_no_of_pairs_column(sequence, results_df)
    generate_position_in_utr_column(sequence, results_df)
    generate_flanking_dinucleotides_columns(sequence, results_df)
    generate_local_au_content_column(sequence, results_df)
    one_hot_encode_match_types(results_df)
    # generate_stable_duplex_column(sequence, results_df, energy_cutoff=-20.0, energy_range=5)
    generate_accessibility_column(sequence, results_df)
    # generate_rnacofold_energy_column(sequence, results_df)
    # generate_rnaduplex_energy_column(sequence, results_df)
    
    return results_df

In [5]:
seq_1m = sequence[:1000000]
results_df = find_matches(seq_1m, targetscan_df)

In [8]:
seq = "ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAAACTGAATCCCATCAAGAAAACTAGTTTCTATTGTATTAGTAACTCAAAATAAAT"

seq = thymine_to_uracil(seq)

results_df2 = modded_run_script(seq)

In [9]:
results_df2.head()

,name,start,end,seed_without_anchor_a,seed_match,match_type,mirna_sequence,ta,3utr_length,3_supplementary_pairing,...,adenine_3end,cytosine_3end,guanine_3end,uracil_3end,5end_au_content_score,3end_au_content_score,m8,seedmatch,a1,accessibility
2,hsa-miR-4282,18,24,AAAUUUU,AAAUUUU,7mer-m8,UCCUGGAUGCAAAUUUUA,4.435,100,0,...,1,0,0,1,0.525869,2.170745,1,1,0,-0.608
3,hsa-miR-548c-3p,20,26,AAUUUUU,AUUUUUA,7mer-A1,GCAAAAGUAAUUGAGAUUUUUG,4.503,100,0,...,1,0,0,1,0.663965,1.898370,0,1,1,-0.600
4,hsa-miR-3163,22,28,UUUUUAU,UUUUAUA,7mer-A1,GUCUUACUGCCCUCAUUUUAUA,4.174,100,0,...,1,1,0,0,0.824221,1.446306,0,1,1,-0.554
5,hsa-miR-568,24,30,UUAUACA,UUAUACA,7mer-m8,GUGUGUAUACAUUUAUACAU,3.370,100,0,...,1,0,1,0,1.015130,1.482553,1,1,0,-1.194
6,hsa-miR-4742-3p,26,32,UAUACAG,AUACAGA,7mer-A1,CUGCAGGCAAAGGAGAAUACAGA,3.048,100,0,...,1,1,0,0,1.251241,1.617539,0,1,1,-1.150


In [27]:
modded_run_script(seq_1m)

,name,start,end,seed_without_anchor_a,seed_match,match_type,mirna_sequence,ta,3utr_length,3_supplementary_pairing,...,adenine_3end,cytosine_3end,guanine_3end,uracil_3end,5end_au_content_score,3end_au_content_score,m8,seedmatch,a1,accessibility
0,hsa-miR-452-5p,11450,11456,CAACAGU,AACAGUA,7mer-A1,UCAGUUUCCUCUGCAAACAGUU,3.186,1000000,0,...,0,1,0,1,0.000000,1.138653,0,1,1,-1.500
1,hsa-miR-4676-3p,11450,11456,CAACAGU,AACAGUA,7mer-A1,AAGAGCCAGUGGUGAAACAGUG,3.186,1000000,0,...,0,1,0,1,0.000000,1.138653,0,1,1,-1.500
2,hsa-miR-892c-3p,11450,11456,CAACAGU,AACAGUA,7mer-A1,UCCACUCAGAAAGGAAACAGUG,3.186,1000000,0,...,0,1,0,1,0.000000,1.138653,0,1,1,-1.500
3,hsa-miR-1278,11452,11458,CAGUACU,CAGUACU,7mer-m8,AUAGAUGAUAUGCACAGUACUA,3.036,1000000,0,...,0,0,2,0,0.333333,1.018625,1,1,0,-2.224
4,hsa-miR-4712-5p,11454,11460,GUACUGG,GUACUGG,7mer-m8,GAAAUGAGAGACCUGUACUGGA,2.831,1000000,0,...,0,1,1,0,0.450000,1.284020,1,1,0,-2.983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183098,hsa-miR-3197,999984,999990,GCGCCUC,GCGCCUC,7mer-m8,CGCCUUUCCGAGCCUGCGCCUCC,2.558,1000000,0,...,0,2,0,0,0.408489,0.166667,1,1,0,-6.741
183099,hsa-miR-6825-5p,999987,999993,CCUCCCC,CCUCCCC,7mer-m8,AUGCUGACUCCACACCUCCCCA,2.948,1000000,0,...,0,0,1,1,0.333015,0.333333,1,1,0,-6.966
183100,hsa-miR-6777-5p,999988,999994,CUCCCCG,CUCCCCG,7mer-m8,UCCACCACUGCCUGACUCCCCGU,2.775,1000000,0,...,0,0,1,1,0.314089,0.500000,1,1,0,-7.186
183101,hsa-miR-6889-5p,999988,999994,CUCCCCG,CUCCCCG,7mer-m8,AUUCCGGACCCCAGACUCCCCGA,2.775,1000000,0,...,0,0,1,1,0.314089,0.500000,1,1,0,-7.186


for 1m bp's:

- find matches takes 7 secs
- modded run script takes 150 secs (7 of those is finding matches)
- accessibility calculation (RNAplfold) takes approx. 2 minutes